In [24]:
import os
import json
import glob
import pandas as pd


In [150]:
RUN_ENV = 'PC' # 'PC' or 'COLAB'

# Load Data by uploading json files maded by ning (need to repeat every time b4 u start)

In [151]:
if RUN_ENV == 'COLAB':
    #json文件，地址在github库MF_KOC\Data\raw\, 导入上传文件为data frame格式
    uploaded_creator = files.upload()
    df_creator = pd.read_json(next(iter(uploaded_creator)))
    uploaded_content = files.upload()
    df_contents = pd.read_json(next(iter(uploaded_content)))
elif RUN_ENV == 'PC':
    #读取本地json文件
    df_creator = pd.read_json(r'..\Data\raw\creator_total.json')
    df_contents = pd.read_json(r'..\Data\raw\contents_total.json')

In [152]:
df_creator.shape

(102, 11)

In [153]:
df_contents.shape

(15941, 21)

# Convert columns to the right dtype, and remove the irrelevant columns

In [154]:
# Define the desired data types for each column of df_creator
desired_dtypes = ['string']*6 + ['int32']*3 + ['string'] +  ['int64']
df = df_creator
# Assign the data types to the columns
for col, dtype in zip(df.columns, desired_dtypes):
    df[col] = df[col].astype(dtype)

In [155]:
df_creator.dtypes

user_id           string[python]
nickname          string[python]
gender            string[python]
avatar            string[python]
desc              string[python]
ip_location       string[python]
follows                    int32
fans                       int32
interaction                int32
tag_list          string[python]
last_modify_ts             int64
dtype: object

In [156]:
# Define the desired data types for each column of df_contents
desired_dtypes = ['object']*2 + ['string']*3 + ['int64']*2 + ['string']*3 +  ['int32']*4 + ['string']*3 + ['int64'] + ['string']*3
df = df_contents
# Assign the data types to the columns
for col, dtype in zip(df.columns, desired_dtypes):
    df[col] = df[col].astype(dtype)

# Treat the df_contents table 

### 7.2, 7.5, 7.6, 7.8, 7.9, 

In [ ]:
# remove columns, ['video_url', 'nickname', 'avatar', 'ip_location', 'note_url', 'source_keyword', 'xsec_token']
df_contents = df_contents.drop(columns=['video_url', 'nickname', 'avatar', 'ip_location', 'note_url', 'source_keyword', 'xsec_token'], inplace=True)

In [158]:
df_contents.dtypes

note_id                     object
type                        object
title               string[python]
desc                string[python]
time                         int64
last_update_time             int64
user_id             string[python]
liked_count                  int32
collected_count              int32
comment_count                int32
share_count                  int32
image_list          string[python]
tag_list            string[python]
last_modify_ts               int64
dtype: object

### 7.1 Type: change the normal to 0, video to 1

In [159]:
df_contents['type'].value_counts()


type
normal    12415
video      3526
Name: count, dtype: int64

In [160]:
# Replace 'normal' with 0 and 'video' with 1 in the 'type' column
df_contents['type'] = df_contents['type'].map({'normal': 0, 'video': 1}).astype('int8')

# Change the column name 'type' to 'content_type_video'
df_contents.rename(columns={'type': 'content_type_video'}, inplace=True)

### 7.7 [Xinbo]Replace image urls with image count

In [161]:
# Create a new column 'image_count' by counting the URLs in 'image_list'
df_contents['image_count'] = df_contents['image_list'].apply(
    lambda x: len(x.split(',')) if isinstance(x, str) and x.strip() != '' else 0
)

# Drop the original 'image_list' column
df_contents.drop('image_list', axis=1, inplace=True)


In [162]:
# Convert 'image_count' to int8
df_contents['image_count'] = df_contents['image_count'].astype('int8')

### 7.12 [Xinbo]note的互动总数(帮助确定爆款笔记)

In [163]:
df_contents['interaction_count'] = df_contents[['liked_count', 'collected_count', 'share_count']].sum(axis=1)


### 5. [Ning]筛选爆款hot note

In [164]:
# A. 对于千粉级creator(粉丝数>1000):
# 	If a note 互动总数+分享数 > 粉丝数/10 + 50 (用当前粉丝数, 忽略粉丝成长曲线)
# B. 对于素人级creator(1000>粉丝数):
# 	If a note 互动总数+分享数 > 150
def get_hot_note(df_creator,df_contents):
    """
    提取爆款笔记，并在原有完整笔记数据表中添加一列 hot_note, 用0或1标注是否为爆款笔记

    Args:
        df_creator (DataFrame): 完整用户数据.
        df_contents (DataFrame): 完整笔记数据.
    Return:
        df_hot_content (DataFrame): 爆款笔记数据
        df_contents (DataFrame): 添加后的笔记数据.
    """
    merged_df = pd.merge(
    df_creator[['user_id', 'fans']],
    df_contents,
    on='user_id'
    )
    maskA = (
        (merged_df['fans'] >= 1000) &
        ((merged_df['liked_count'] + merged_df['collected_count'] +
          merged_df['comment_count'] + merged_df['share_count']) >
         (merged_df['fans'] / 10 + 50))
    )

    maskB = (
        (merged_df['fans'] < 1000) &
        ((merged_df['liked_count'] + merged_df['collected_count'] +
          merged_df['comment_count'] + merged_df['share_count']) > 150)
    )

    mask = maskA | maskB
    hot_content = merged_df[mask]

    df_contents['hot_note'] = 0
    hot_list = hot_content['note_id'].values.tolist()
    for i in range(len(df_contents)):
      rows = df_contents.iloc[i]
      note_id = rows['note_id']
      if note_id in hot_list:
        df_contents.loc[df_contents['note_id']==note_id,'hot_note']=1

    return df_contents, hot_content

# Get hot notes and add a new column to df_contents
df_contents, df_hot_content = get_hot_note(df_creator, df_contents)

In [165]:
df_contents.head()

,note_id,content_type_video,title,desc,time,last_update_time,user_id,liked_count,collected_count,comment_count,share_count,tag_list,last_modify_ts,image_count,interaction_count,hot_note
0,67d0d605000000002903d86a,0,有没有和我一样【吃茶三千】一喝一个不吱声的,连地址都不想写了[笑哭R] 虽然环境还不错，颜值也不错 但我感觉自己完全拔草了 尝了mang...,1741739525000,1741739526000000000,576d3bde82ec3952ff40c5e1,5,2,10,3,"达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店",1741740908175,1,10,0
1,67c8960f000000002503ff5f,0,【达拉斯·玩】超治愈系手工体验！MUMU Garden,"📍301 W Parker Rd #208[话题]#, Plano, TX 75023（快乐...",1741198863000,1741198864000000000,576d3bde82ec3952ff40c5e1,47,23,7,37,"208,达拉斯生活,达拉斯探店,达拉斯,达拉斯吃喝玩乐,达拉斯手工,达拉斯周末,达拉斯周边,...",1741740908248,14,107,0
2,67b2c2f0000000002602e61d,0,【达拉斯·吃】米其林推荐川菜主厨空降达拉斯？,"📍2528 Old Denton Rd, Carrollton, TX 75006 听说米其...",1739768560000,1739804941000000000,576d3bde82ec3952ff40c5e1,85,80,38,155,"达拉斯美食,达拉斯生活,Texas,达拉斯,达拉斯周边美食,达拉斯探店,达拉斯川菜,carr...",1741740908264,15,320,1
3,67a926770000000029011c49,0,【达拉斯·吃】许留山这几碗糯唧唧绝了！,"Hui Liu Shan 📍2540 Old Denton Rd Ste 310, Carr...",1739138679000,1739155924000000000,576d3bde82ec3952ff40c5e1,48,20,5,15,"达拉斯,德州,达拉斯美食,德州美食,达拉斯探店,达拉斯生活,许留山,达拉斯甜品,达拉斯网红甜...",1741740908290,5,83,0
4,67a3d27600000000290360e3,0,【达拉斯·喝】Frosty Fox奶茶礼盒开箱,大过年的没想到收到一个奶茶礼盒做开箱分享！新年快乐[大笑R] Frosty Fox是一家从佛...,1738789494000,1738803429000000000,576d3bde82ec3952ff40c5e1,15,4,4,2,"达拉斯奶茶,达拉斯网红奶茶,达拉斯美食,德州美食,达拉斯生活,德州生活,frostyfox,...",1741740908312,7,21,0


### 7.13 [Yucheng]笔记在EDA图的时间戳可以设定为Time和last_update_time的一个函数。提取时间戳中的 年月日、小时、星期几、是否节假日 等特征

In [166]:
def extract_time_features(df):
    """
    Extracts time-related features from a DataFrame.

    Args:
        df: DataFrame with a 'combined_timestamp' column representing timestamps.

    Returns:
        DataFrame with added time-related features.
    """

    # Convert to datetime objects if not already
    if not pd.api.types.is_datetime64_any_dtype(df['combined_timestamp']):
        df['combined_timestamp'] = pd.to_datetime(df['combined_timestamp'], unit='ms')

    df['year'] = df['combined_timestamp'].dt.year
    df['month'] = df['combined_timestamp'].dt.month
    df['day'] = df['combined_timestamp'].dt.day
    df['hour'] = df['combined_timestamp'].dt.hour
    df['dayofweek'] = df['combined_timestamp'].dt.dayofweek  # Monday=0, Sunday=6

    # Placeholder for holiday feature – needs external holiday data
    df['is_holiday'] = 0  # Initialize to 0 (not a holiday)
    # You'll need to replace this with actual holiday detection logic.

    return df

# Example usage (assuming df_contents has 'combined_timestamp' column)
df_contents = extract_time_features(df_contents)
print(df_contents[['combined_timestamp', 'year', 'month', 'day', 'hour', 'dayofweek', 'is_holiday']].head())


KeyError: 'combined_timestamp'

In [ ]:
def timestamp_function(row):
    """
    Example function to combine 'time' and 'last_update_time'.
    You can customize this to your specific needs.
    """
    time_val = row['time']
    last_update_time_val = row['last_update_time']

    #Return the later of the two timestamps
    return max(time_val, last_update_time_val)

# Apply the function to create a new column
df_contents['combined_timestamp'] = df_contents.apply(timestamp_function, axis=1)

# Example usage to show the new column:
print(df_contents[['time', 'last_update_time', 'combined_timestamp']].head())


                        time    last_update_time  combined_timestamp
0 1970-01-01 00:29:01.739525 2025-03-12 00:32:06 2025-03-12 00:32:06
1 1970-01-01 00:29:01.198863 2025-03-05 18:21:04 2025-03-05 18:21:04
2 1970-01-01 00:28:59.768560 2025-02-17 15:09:01 2025-02-17 15:09:01
3 1970-01-01 00:28:59.138679 2025-02-10 02:52:04 2025-02-10 02:52:04
4 1970-01-01 00:28:58.789494 2025-02-06 00:57:09 2025-02-06 00:57:09


# Treat the df_creator table 

6.29 [Ning]新加hot note count (请参考上面高亮的 5, 分两类creator, 但需要完成content表)\
6.30 [Ning]新加total share counts from hot notes only(同上)\
6.31 [Ning]新加最近一次发帖到抓取时的天数(floor取整), last_note2now\
6.32 [Ning]新加最近一次发hot note到抓取时的天数(floor取整), last_hot_note2now

In [ ]:
def timestamp_diff_to_days(t1, t2):
    """
    计算毫米级时间戳的差值，并转化为天数，向下取整

    Args:
        t2 (int): 第二时间.
        t1 (int): 第一时间.
    Return:
        days (int): 差值天数
    """
    delta_ms = t2 - t1
    days = delta_ms // 86400000
    return int(days)
def add_columns_creator_Ning(df_creator,df_contents,hot_content):
    """
    在用户数据中添加 hot_note_count, total_share_counts_hot_only, last_note2now, last_hot_note2now

    Args:
        df_creator (DataFrame): 完整用户数据.
        df_contents (DataFrame): 完整笔记数据.
        hot_content (DataFrame): 爆款笔记数据.
    Return:
        df_creator (DataFrame): 添加完成后的用户数据
    """
    df_creator['hot_note_count'] = 0
    df_creator['total_share_counts_hot_only'] = 0
    df_creator['last_note2now'] = 0
    df_creator['last_hot_note2now'] = 0
    for i in range(len(df_creator)):
      rows = df_creator.iloc[i]
      user_id = rows['user_id']
      current_hot_content = hot_content[hot_content['user_id']== user_id]
      df_creator.loc[df_creator['user_id'] == user_id, 'hot_note_count'] = len(current_hot_content)
      df_creator.loc[df_creator['user_id'] == user_id, 'total_share_counts_hot_only'] = current_hot_content['share_count'].sum()
      time_last_note = df_contents.loc[df_contents['user_id']== user_id]['time'].max()
      time_craw_note = df_contents.loc[(df_contents['user_id']== user_id) & (df_contents['time']== time_last_note), 'last_modify_ts'].max()
      time_last_hot = hot_content.loc[hot_content['user_id']== user_id]['time'].max()
      time_craw_hot = hot_content.loc[(hot_content['user_id']== user_id) & (hot_content['time']== time_last_hot), 'last_modify_ts'].max()
      if len(current_hot_content) != 0:
        df_creator.loc[df_creator['user_id'] == user_id, 'last_note2now'] = timestamp_diff_to_days(time_last_note, time_craw_note)
        df_creator.loc[df_creator['user_id'] == user_id, 'last_hot_note2now'] = timestamp_diff_to_days(time_last_hot, time_craw_hot)
    return df_creator


## Add Interaction Counts
Refine Creator (add sums)
- Get the total amount of `liked`, `collected` and `shared`, and set their data type to integer
- Get the total number of posts as `note_count`, and set its data type to integer

In [9]:
# Ensure numeric columns in the content df are in the proper format
for col in ['liked_count', 'collected_count', 'comment_count', 'share_count']:
    df_contents[col] = pd.to_numeric(df_contents[col], errors='coerce')

# Group content df by user_id and sum the counts
agg_counts = df_contents.groupby('nickname')[['liked_count', 'collected_count', 'comment_count', 'share_count']].sum().reset_index()

# Merge the aggregated counts into the creator df by aligning on user_id
df_creator = pd.merge(df_creator, agg_counts, on='nickname', how='left')
# Convert the data type to integer
for col in ['liked_count', 'collected_count', 'comment_count', 'share_count']:
    df_creator[col] = df_creator[col].convert_dtypes()


In [10]:
note_counts = df_contents.groupby("nickname")["note_id"].count().reset_index(name="note_count")
df_creator = pd.merge(df_creator, note_counts, on="nickname", how="left")
df_creator["note_count"] = df_creator["note_count"].fillna(0).astype(int)

In [11]:
df_creator.head(2)

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,tag_list,last_modify_ts,liked_count,collected_count,comment_count,share_count,note_count
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,女,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰,美国,837,1653,11636,"{""info"": null, ""location"": ""美国"", ""profession"":...",1741740734045,6443,4951,2847,4433,146
1,58b325ce50c4b442ae017a30,小温柔Sylvia,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,"{""info"": null, ""location"": ""北京海淀""}",1741746441631,7290,3354,1713,1818,142


## Processing Tag list
There are four different categories in tag lists: info, location, profession and college.

Processing tag lists as
- Creating a columns as `zodiac` and `age` for 星座 and 年龄。
- Note that there are 5 missing values in the `ip_location` column where the `location` column is not missing. I plan to use the `location` data to impute these missing `ip_location` values. Additionally, there are 22 cases where `ip_location` and `location` do not match (among the non-missing location entries), which include typos, missing values, and additional details present in the `location` field. (For detailed location info, we still need to discuss how to deal with it.)
- For `college`, there are only few KOC having this information, so I append it behind `desc` connected by '|'.
- For `prefession`, also append it behind `desc`.

In [14]:
tag_process = df_creator['tag_list'].apply(lambda x: json.loads(x))

# Extract unique column names from all dictionaries
distinct_columns = set()
tag_process.apply(lambda x: distinct_columns.update(x.keys()))

# Display results
print("Distinct Column Names:", distinct_columns)

Distinct Column Names: {'profession', 'college', 'info', 'location'}


In [15]:
for col in distinct_columns:
    df_creator[col] = tag_process.apply(lambda x: x.get(col, None))

**info**

In [16]:
# create new column as zodiac
mask = df_creator['info'].str.contains('座', na = False)
df_creator['zodiac'] = df_creator["info"].mask(~mask, None)

In [17]:
mask = df_creator['info'].str.contains('岁', na = False)
df_creator['age'] = df_creator["info"].mask(~mask, None)
df_creator.loc[df_creator['age'].notna(),'age'] = df_creator.loc[df_creator.age.notna(),'age'].str.slice(stop = 2).astype(int)

**location**

In [18]:
df_creator['location'] = df_creator['location'].replace('UnitedStates','美国')
df_creator['location'] = df_creator['location'].replace('美国得克萨斯','美国TX')
df_creator['location'] = df_creator['location'].replace('UnitedStates...','美国')
df_creator['location'] = df_creator['location'].replace('海外美国','美国')

#### Note:
- ip_location: directly from profile page
- location: separated from user self-added tags

In [19]:
df_creator[['ip_location','location']][((~df_creator['location'].isna()) & (df_creator['location'] != df_creator["ip_location"]))]

,ip_location,location
1,美国,北京海淀
5,美国,美国TX
20,,美国
30,美国,美国NY
31,墨西哥,美国TX
37,美国,美国TX
38,美国,美国TX
49,美国,巴巴多斯
54,美国,美国TX
55,,美国TX


### Impute `ip_location` by `location`

In [20]:
df_creator.loc[((df_creator['ip_location'].isna())&(df_creator['location'].notna())),'ip_location'] = df_creator.loc[((df_creator['ip_location'].isna())&(df_creator['location'].notna())),'location'].str.slice(stop = 2)

In [21]:
df_contents[df_contents.nickname.isin(df_creator[df_creator.ip_location.isna()]['nickname'])][['nickname','ip_location']] \
.groupby('nickname').ip_location.value_counts(dropna = False)

,,count
nickname,ip_location,


In [22]:
df_creator.loc[(df_creator['ip_location'].isna()),'ip_location'] = '美国'

In [23]:
df_creator['location'] = df_creator['location'].replace('美国','').replace('',None)

In [ ]:
df_creator[['ip_location','location']][((~df_creator['location'].isna()) & (df_creator['location'] != df_creator["ip_location"]))]

,ip_location,location
0,美国,巴巴多斯
13,美国,美国NY
14,墨西哥,美国TX
20,美国,美国TX
21,美国,美国TX
37,美国,美国TX
38,,美国TX
39,美国,美国TX
49,美国,美国TX
52,美国,北京海淀


For the detailed location description, still need discuss.

**college**

In [24]:
df_creator.college.notna().sum()

np.int64(7)

In [25]:
df_creator.loc[df_creator.college.notna(),'desc'] = df_creator.desc[df_creator.college.notna()] + '|' + df_creator.college[df_creator.college.notna()]

In [26]:
df_creator.desc[df_creator.college.notna()]

,desc
29,𝙀𝙣𝙟𝙤𝙮 𝙡𝙞𝙛𝙚 ʚ🧸ྀིɞ 📍Dallas\n🦋ig: yvonneyifanhe\n...
38,德州🇺🇸 | JHUer DC🇺🇸 |北京🇨🇳\nESTJ | 双子 | 美食旅游探店小能手...
52,白羊INFP-A\n欢迎来到我的朋友圈|南卫理公会大学
54,杭州|PSU|Cornell➡️Dallas\n00.06.12 INFJ \n🎵🍽️📷🖤|...
69,📷🥑🏝️👨🏻‍⚕️🇺🇸UT Southwestern➡️Stanford Medicine|...
73,"Speak Texan, Cantonese, Mandarin\n愛上德州Texas!\n..."
100,｜35+ 会计打工人\n｜喜爱跳舞/追剧/分享生活有用信息/偶尔健身/男宝妈|悉尼麦考瑞大学


**profession**

In [27]:
df_creator.profession.notna().sum()

np.int64(28)

In [28]:
df_creator.profession[df_creator.profession.notna()].value_counts()

,count
profession,
旅行博主,6
美食博主,5
探店博主,4
料理博主,2
vlog博主,2
家居博主,2
摄影博主,2
物流从业者,1
建筑工人,1


I will merge the profession into `desc` also.

In [29]:
df_creator.loc[df_creator.profession.notna(),'desc'] = df_creator.desc[df_creator.profession.notna()] + '|' + df_creator.profession[df_creator.profession.notna()]

In [30]:
df_creator.loc[df_creator.profession.notna(),'desc'][0]

'慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰|探店博主'

## Feature Engineering

**KOC level**
- `ff_ratio`: ratio calculated by fans over follows
- `video_ratio`: ratio calculated by number of video posts over total number posts
- `account_age`: the time length in days between the newest post and oldest post

In [31]:
df_creator['ff_ratio'] = df_creator.fans / df_creator.follows
video_ratio = df_contents.groupby(['nickname','type']).size().unstack().fillna(0).reset_index()
video_ratio['video_ratio'] = video_ratio.video / (video_ratio.video + video_ratio.normal)
df_creator = pd.merge(df_creator, video_ratio[['nickname', 'video_ratio', 'normal']], on = 'nickname')
df_contents['time'] = pd.to_datetime(df_contents.time, unit = 'ms')
time_length = df_contents.groupby('nickname') \
    .agg(oldest = ('time', 'min'), newest = ('time', 'max')).reset_index()
time_length['age_koc'] = time_length.newest - time_length.oldest
time_length['age_koc'] = time_length.age_koc.dt.days
df_creator = pd.merge(df_creator, time_length[['nickname', 'age_koc']], on = 'nickname')

In [32]:
df_creator = df_creator.drop(columns = ['college','tag_list','profession','info'])

In [33]:
df_creator.head(1)

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,last_modify_ts,...,comment_count,share_count,note_count,location,zodiac,age,ff_ratio,video_ratio,normal,age_koc
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,女,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰|探店博主,美国,837,1653,11636,1741740734045,...,2847,4433,146,None,None,None,1.97491,0.013699,144.0,2054


In [34]:
df_creator.describe()

,follows,fans,interaction,last_modify_ts,liked_count,collected_count,comment_count,share_count,note_count,ff_ratio,video_ratio,normal,age_koc
count,102.000000,102.000000,102.000000,1.020000e+02,102.0,102.0,102.0,102.0,102.000000,102.000000,102.000000,102.00000,102.000000
mean,494.578431,1267.078431,15992.794118,1.741155e+12,8883.352941,3719.009804,1836.568627,2049.892157,150.490196,33.864857,0.216532,116.72549,1257.401961
std,704.586789,1598.743799,23452.095995,8.898437e+09,16053.605159,4506.916549,2363.340217,2175.625186,153.756123,239.198580,0.231589,119.14989,701.044797
min,3.000000,69.000000,1069.000000,1.652198e+12,336.0,116.0,62.0,23.0,11.000000,0.178448,0.000000,1.00000,12.000000
25%,94.750000,390.500000,4036.500000,1.741822e+12,2051.25,1096.25,543.25,661.5,54.250000,1.084638,0.043012,41.25000,820.250000
50%,219.000000,708.000000,7174.000000,1.741974e+12,4047.0,1999.5,1064.0,1300.0,97.000000,3.491964,0.125262,79.50000,1277.000000
75%,500.500000,1550.000000,14676.500000,1.742098e+12,7244.0,4476.25,2253.0,2692.5,175.500000,9.674224,0.338652,147.00000,1605.000000
max,3749.000000,8955.000000,147344.000000,1.742580e+12,127240.0,25255.0,17114.0,11566.0,781.000000,2417.666667,0.980769,702.00000,3089.000000


In [35]:
df_creator.isna().sum()

,0
user_id,0
nickname,0
gender,9
avatar,0
desc,0
ip_location,0
follows,0
fans,0
interaction,0
last_modify_ts,0


### Impute Gender

In [36]:
df_creator.gender.value_counts(dropna = False)

,count
gender,
女,83
男,10
None,9


In [37]:
df_creator[df_creator.gender.isna()]

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,last_modify_ts,...,comment_count,share_count,note_count,location,zodiac,age,ff_ratio,video_ratio,normal,age_koc
13,663ab3fc0000000007005c31,朱迪日常,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,探店测评|好物分享\n常驻Dalls TX ～,美国,21,189,2478,1742053971642,...,608,913,73,None,None,None,9.000000,0.123288,64.0,218
18,62f0659d000000001f015932,达拉斯幸福家Hannah,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,HannahRealtor|家居博主,美国,276,1875,6475,1742080944296,...,1080,2724,127,None,None,None,6.793478,0.456693,69.0,554
22,67865b1a000000000803eeb0,Dallas Miranda,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,Ig & YT: dallasmirandaklein \nOver 400k friend...,,3,7253,24271,1741678589024,...,489,23,15,None,None,None,2417.666667,0.933333,1.0,17
34,6785e363000000000801d306,Styleindallas,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,"Fashion, beauty , aesthetic 📸\nMarried 💍\nInst...",美国,32,246,6633,1741811076247,...,62,23,52,None,None,None,7.687500,0.980769,1.0,55
71,6303b99d0000000012001f58,北极熊的美国生活笔记,None,https://sns-avatar-qc.xhscdn.com/avatar/64dd4d...,🇺🇸17年\n🏠 南京 ❄️波士顿 🌧️西雅图 🌞达拉斯\n你的北美生活博主\n摄影+旅游👉...,美国,49,1490,28504,1742440998789,...,4871,6105,126,None,None,None,30.408163,0.023810,123.0,532
77,6104251600000000010052ab,JoanneJJJ,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,👩🏻‍💻四大 · CPA\n要开心·要洒脱·要爱自己·要好好吃饭·要好好睡觉\nig: ba...,,48,404,2381,1742453206427,...,150,205,31,None,双子座,None,8.416667,0.129032,27.0,408
79,676f0595000000001801da5d,Xena,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,🎮 🎨💪🏽📺📚🧠💄🐶🌵🥾♏️\n\nHi Everyone \nWelcome to my ...,,200,4528,99941,1742519641927,...,3033,27,24,None,天蝎座,None,22.640000,0.625000,9.0,14
94,6613fa0f000000000303117f,Coco,None,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,记录美国生活,美国,64,220,1342,1742076635638,...,640,179,64,None,None,None,3.437500,0.203125,51.0,207
99,5c4be6f500000000100050c6,小fafa,None,https://sns-avatar-qc.xhscdn.com/avatar/62f3f2...,📍TX🇺🇸\n吃喝｜遛娃｜摄影｜旅行\n旅行笔记已分类📁|摄影博主,美国,180,1732,30397,1742407398300,...,5455,4286,372,None,None,None,9.622222,0.072581,345.0,1803


No need to use `XHS_KOC_List.csv` to impute the gender. They should all be imputed as Female

In [53]:
# df_xhs_koc_sex = pd.read_csv('https://docs.google.com/spreadsheets/d/1D5irT_7hQ8TKDmG7wnUaIZvkMZ-2M5tE6esM6_lVe2I/edit?gid=0#gid=0', skipinitialspace= True, usecols=["Sex"], header=1)

ValueError: Usecols do not match columns, columns expected but not found: ['Sex']

In [ ]:
# df_xhs_koc.Sex.isna().sum()

There are 3 KOC we need to search online to see the gender. For the rest missing gender (5 KOC), we can impute them from the google sheet.

In [ ]:
# pd.merge(df_creator[df_creator.gender.isna()][['nickname']],df_xhs_koc[df_xhs_koc.Sex.isna()][['Name']], left_on = 'nickname', right_on = 'Name')

In [ ]:
# df_xhs_koc.loc[df_xhs_koc.Name.isin(df_creator[df_creator.gender.isna()]['nickname']), 'Sex']

In [54]:
df_creator.loc[df_creator.gender.isna(),'gender'] = '女'

In [55]:
df_creator.gender.value_counts(dropna = False)

,count
gender,
女,92
男,10


给KOC加post_span

In [61]:
post_times = df_contents.groupby('user_id')['time'].agg(['min', 'max']).reset_index()
post_times['post_span'] = post_times['max'] - post_times['min']
print(post_times[['user_id', 'min', 'max', 'post_span']].head())

                    user_id                 min                 max  \
0  54d82c482e1d9366ebfc8ce5 2020-11-22 20:47:10 2025-03-08 05:22:29   
1  5530f8642e1d9365fa9a29fb 2021-05-10 16:54:57 2025-02-28 22:56:16   
2  55481451b203d97e86008c02 2020-12-02 00:14:57 2025-01-25 13:47:58   
3  556f1983f5a26347140b104b 2017-02-19 08:08:58 2025-03-13 22:58:53   
4  558a4a065894460336de85bf 2021-08-11 10:43:04 2025-03-16 04:03:26   

           post_span  
0 1566 days 08:35:19  
1 1390 days 06:01:19  
2 1515 days 13:33:01  
3 2944 days 14:49:55  
4 1312 days 17:20:22  


In [63]:
df_creator = pd.merge(df_creator, post_times, on='user_id', how='left')
df_creator.head()

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,last_modify_ts,...,ff_ratio,video_ratio,normal,age_koc,min_x,max_x,post_span_x,min_y,max_y,post_span_y
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,女,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰|探店博主,美国,837,1653,11636,1741740734045,...,1.974910,0.013699,144.0,2054,2019-07-27 23:00:43,2025-03-12 00:32:05,2054 days 01:31:22,2019-07-27 23:00:43,2025-03-12 00:32:05,2054 days 01:31:22
1,58b325ce50c4b442ae017a30,小温柔Sylvia,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,1741746441631,...,0.316337,0.014085,140.0,2459,2018-06-12 07:38:23,2025-03-07 01:47:51,2459 days 18:09:28,2018-06-12 07:38:23,2025-03-07 01:47:51,2459 days 18:09:28
2,5c0c067c0000000005022f0a,茄茄公主 🍆,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,坐标 达拉斯 美国🇺🇸 来美14 年 家庭成员： 三只毛小孩🐶 喜欢旅游 和探索各地美食🥰,美国,251,258,3607,1741747540369,...,1.027888,0.256757,165.0,1879,2019-12-25 03:42:14,2025-02-15 04:23:29,1879 days 00:41:15,2019-12-25 03:42:14,2025-02-15 04:23:29,1879 days 00:41:15
3,5649379603eb842b7ed00477,Sherry是睿睿呀,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北美生活探店| Chinese Girl in Dallas TX\n吃喝玩乐样样行 | 坐...,美国,1173,1326,28064,1741807240419,...,1.130435,0.075758,61.0,476,2023-11-15 07:27:31,2025-03-05 23:52:07,476 days 16:24:36,2023-11-15 07:27:31,2025-03-05 23:52:07,476 days 16:24:36
4,5d6a927e00000000010068b9,達拉斯里長,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,達拉斯，請多指教👋\n在朝九晚五平淡生活中，\n熱烈又燦爛的挖掘美好✨,美国,353,794,8149,1741814164296,...,2.249292,0.144928,118.0,1262,2021-08-24 23:44:32,2025-02-07 22:07:07,1262 days 22:22:35,2021-08-24 23:44:32,2025-02-07 22:07:07,1262 days 22:22:35


Add post span

In [ ]:
earliest_posts = df_contents.groupby('user_id')['time'].min().reset_index()
earliest_posts.rename(columns={'time': 'first_post_time'}, inplace=True)

# Merge the earliest post time into df_creator based on user_id.
df_creator = pd.merge(df_creator, earliest_posts, on='user_id', how='left')

# Calculate account_length as the difference between last_modify_ts and first_post_time.
df_creator['account_length'] = df_creator['last_modify_ts'] - df_creator['first_post_time']


In [ ]:
df_creator

Add average post time & std

In [ ]:
import numpy as np
def compute_history_stats(times):
    if len(times) <= 1:
        return np.nan, np.nan
    sorted_times = np.sort(times)
    intervals = np.diff(sorted_times)
    return intervals.mean(), intervals.std()

history_stats = df_contents.groupby('user_id')['time'].apply(
    lambda x: compute_history_stats(x.values)
).reset_index()

history_stats[['history_avg', 'history_std']] = pd.DataFrame(history_stats['time'].tolist(), index=history_stats.index)

history_stats.drop(columns=['time'], inplace=True)

df_creator = pd.merge(df_creator, history_stats, on='user_id', how='left')
print(df_creator[['user_id', 'history_avg', 'history_std']].head())


Add counts within 90 days

In [ ]:

threshold_ms = 90 * 24 * 3600 * 1000  # 90 days in ms

# Merge df_contents with the subset of df_creator that has user_id and last_modify_ts.
# Use suffixes so that the creator's timestamp comes out with a unique name.
df_merged = pd.merge(
    df_contents,
    df_creator[['user_id', 'last_modify_ts']],
    on='user_id',
    how='left',
    suffixes=('', '_creator')
)

print("Columns in df_merged:", df_merged.columns.tolist())

# Now, the reference timestamp should be in the column 'last_modify_ts_creator'
# Filter for posts within the 90-day window from each creator's last_modify_ts.
df_recent = df_merged[df_merged['time'] >= (df_merged['last_modify_ts_creator'] - threshold_ms)]

# Aggregate the metrics for these recent posts by user_id.
metrics_90 = df_recent.groupby('user_id').agg(
    liked_90=('liked_count', 'sum'),
    collected_90=('collected_count', 'sum'),
    comment_90=('comment_count', 'sum'),
    share_90=('share_count', 'sum'),
    note_count_90=('note_id', 'count')
).reset_index()

# Merge the 90-day metrics into df_creator.
df_creator = pd.merge(df_creator, metrics_90, on='user_id', how='left')

# For creators with no posts in the last 90 days, fill missing values with 0.
cols_to_fill = ['liked_90', 'collected_90', 'comment_90', 'share_90', 'note_count_90']
df_creator[cols_to_fill] = df_creator[cols_to_fill].fillna(0)

# Display the new columns to verify the result.
print(df_creator[['user_id', 'liked_90', 'collected_90', 'comment_90', 'share_90', 'note_count_90']].head())


In [ ]:
threshold_ms = 180 * 24 * 3600 * 1000  # 90 days in ms

df_merged = pd.merge(
    df_contents,
    df_creator[['user_id', 'last_modify_ts']],
    on='user_id',
    how='left',
    suffixes=('', '_creator')
)

print("Columns in df_merged:", df_merged.columns.tolist())

df_recent = df_merged[df_merged['time'] >= (df_merged['last_modify_ts_creator'] - threshold_ms)]

# Aggregate the metrics for these recent posts by user_id.
metrics_180 = df_recent.groupby('user_id').agg(
    liked_180=('liked_count', 'sum'),
    collected_180=('collected_count', 'sum'),
    comment_180=('comment_count', 'sum'),
    share_180=('share_count', 'sum'),
    note_count_180=('note_id', 'count')
).reset_index()

df_creator = pd.merge(df_creator, metrics_180, on='user_id', how='left')

cols_to_fill = ['liked_180', 'collected_180', 'comment_180', 'share_180', 'note_count_180']
df_creator[cols_to_fill] = df_creator[cols_to_fill].fillna(0)


Columns in df_merged: ['note_id', 'type', 'title', 'desc', 'video_url', 'time', 'last_update_time', 'user_id', 'nickname', 'avatar', 'liked_count', 'collected_count', 'comment_count', 'share_count', 'ip_location', 'tag_list', 'last_modify_ts', 'note_url', 'source_keyword', 'xsec_token', 'image_count', 'combined_time', 'last_modify_ts_creator']


TypeError: Invalid comparison between dtype=datetime64[ns] and ndarray